In [1]:
#Importacion de librerias
import bempp.api
import dolfin
import numpy as np 
import time
from readpqr import *
start = time.time()

In [2]:
#Datos Principales. 
em = 4.     #[-] Permitividad electrica interior.
es = 80.    #[-] Permitividad electrica exterior.
k = 0.125   #[1/A] Inverso de la longitud de Debey-Huckel del fluido.

#Ensablaje de los operadores de frontera: #'fmm' para moleculas con un gran mayor numero de vertices.
#'default_nonlocal' para moleculas con un pequeño numero de vertices.
Assemble = 'default_nonlocal' 

In [3]:
#Datos de la posicion en el espacio, carga y radios de los atomos de la molecula.
PC,Q,R = readpqr('PQR/Sphere5Q3.pqr')  #Eleccion del "pqr" de la molecula

In [4]:
#Generar la malla volumetrica de la molecula
from dolfin import Mesh
mesh = Mesh("Mallas_V/outputTetMeshSphere5R0.xml") #Creacion de la malla volumetrica

In [5]:
#Generar espacios de funcionales del potencial en Fem y su derivada en Bem
from bempp.api.external import fenics

fenics_space = dolfin.FunctionSpace(mesh, "CG", 1) #Potencial electrostatico en la interfaz y dominio del soluto
trace_space, trace_matrix = \
    fenics.fenics_to_bempp_trace_data(fenics_space) # Espacio de la traza para trabajar en BEM y FEM simultaneamente.
bempp_space = bempp.api.function_space(trace_space.grid, "DP", 0) #Derivada del potencial electrostatico en la interfaz.

print("FEM dofs: {0}".format(mesh.num_vertices()))
print("BEM dofs: {0}".format(bempp_space.global_dof_count))
print("TRA dofs: {0}".format(trace_space.global_dof_count))

FEM dofs: 367
BEM dofs: 576
TRA dofs: 290


In [6]:
#Generar operadores de frontera de Fem y Bem
I1 = bempp.api.operators.boundary.sparse.identity(trace_space, bempp_space, bempp_space) #1
mass = bempp.api.operators.boundary.sparse.identity(bempp_space, bempp_space, trace_space) #1
if k==0:
    K1 = bempp.api.operators.boundary.laplace.double_layer(trace_space, bempp_space, bempp_space, assembler=Assemble) #K
    V1 = bempp.api.operators.boundary.laplace.single_layer(bempp_space, bempp_space, bempp_space, assembler=Assemble) #V 
else:
    K1 = bempp.api.operators.boundary.modified_helmholtz.double_layer(trace_space, bempp_space, bempp_space, k, assembler=Assemble) #K
    V1 = bempp.api.operators.boundary.modified_helmholtz.single_layer(bempp_space, bempp_space, bempp_space, k, assembler=Assemble) #V 

In [7]:
#Definir el espacio funcional de Dolfin
u = dolfin.TrialFunction(fenics_space)
v = dolfin.TestFunction(fenics_space)

In [8]:
#Construccion del vector derecho
# EL rhs en Ωm(FEM)
rhs_f = dolfin.Constant(0.0)
rhs_fem = dolfin.assemble(rhs_f*v*dolfin.dx)
#Creacion de funcion del potencial de Coulomb   
for i in range(len(PC)):
    delta = dolfin.PointSource(fenics_space, dolfin.Point(PC[i]), Q[i]/em)
    delta.apply(rhs_fem)
# El rhs en Ωs(BEM)
rhs_bem = np.zeros(bempp_space.global_dof_count)
# La combinacion de rhs
rhs = np.concatenate([rhs_fem, rhs_bem])

In [9]:
#Construccion matriz izquierda 2x2
from bempp.api.assembly.blocked_operator import BlockedDiscreteOperator
from bempp.api.external.fenics import FenicsOperator
from scipy.sparse.linalg.interface import LinearOperator
blocks = [[None,None],[None,None]]

trace_op = LinearOperator(trace_matrix.shape, lambda x:trace_matrix*x)
A = FenicsOperator((dolfin.inner(dolfin.nabla_grad(u),
                                 dolfin.nabla_grad(v)) ) * dolfin.dx)  
#Posicion de la matriz 2x2#
blocks[0][0] = A.weak_form()  #A
blocks[0][1] = -trace_matrix.T * mass.weak_form().A  #-ML
blocks[1][0] = (0.5 * I1 - K1).weak_form() * trace_op  #0.5-K
blocks[1][1] = V1.weak_form()*(em/es)  #V

blocked = BlockedDiscreteOperator(np.array(blocks))

In [10]:
#Creacion del precondicionador Block Diagonal para FEM/BEM
from bempp.api.assembly.discrete_boundary_operator import InverseSparseDiscreteBoundaryOperator
from scipy.sparse.linalg import LinearOperator
from scipy.sparse import diags

P1 = diags(1./blocked[0,0].A.diagonal())
P2 = InverseSparseDiscreteBoundaryOperator(
    bempp.api.operators.boundary.sparse.identity(
        bempp_space, bempp_space, bempp_space).weak_form())

def apply_prec(x):
    """Apply the block diagonal preconditioner"""
    m1 = P1.shape[0]
    m2 = P2.shape[0]
    n1 = P1.shape[1]
    n2 = P2.shape[1]
    
    res1 = P1.dot(x[:n1])
    res2 = P2.dot(x[n1:])
    return np.concatenate([res1, res2])

p_shape = (P1.shape[0] + P2.shape[0], P1.shape[1] + P2.shape[1])
P = LinearOperator(p_shape, apply_prec, dtype=np.dtype('complex128'))

#Se resuelve la solucion de la Ecuacion matricial Ax=B
#Contador de iteraciones
it_count = 0
def count_iterations(x):
    global it_count
    it_count += 1
    if (it_count / 10000) == (it_count // 10000):
        print(it_count,x)
        
# Solucion por GMRES
from scipy.sparse.linalg import gmres
start1 = time.time()
soln, info = gmres(blocked, rhs, M=P, callback=count_iterations,tol=1e-6)  
end1 = time.time() 
curr_time1 = (end1 - start1)
  
soln_fem = soln[:mesh.num_vertices()]
soln_bem = soln[mesh.num_vertices():]

print("Numero de iteraciones de GMRES: {0}".format(it_count))
print("Tiempo total en GMRES: {:5.2f} [s]".format(curr_time1))

/home/mauricioguerrero/Software/miniconda3/envs/bempp_prod/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:318: SparseEfficiencyWarning: splu requires CSC matrix format
  warn('splu requires CSC matrix format', SparseEfficiencyWarning)


Numero de iteraciones de GMRES: 240
Tiempo total en GMRES:  0.21 [s]


In [11]:
#Calcula todo el dominio global del potencial en la interfaz a partir del potencial de FEM y la derivada de BEM calculado
# Solucion para datos de Dirichlet 
dirichlet_data = trace_matrix * soln_fem
dirichlet_fun = bempp.api.GridFunction(trace_space, coefficients=dirichlet_data)
# Solucion para datos de Neumann 
neumann_fun = bempp.api.GridFunction(bempp_space, coefficients=soln_bem)

In [12]:
#Calculo del potencial en la posicion de los atomos de la molecula
slpF = bempp.api.operators.potential.laplace.single_layer(bempp_space, np.transpose(PC)) 
dlpF = bempp.api.operators.potential.laplace.double_layer(trace_space, np.transpose(PC))
uF = slpF * neumann_fun - dlpF * dirichlet_fun 

#Resultado de la energia de solvatacion por atomo
q_uF = 0
for i in range(len(PC)):
    Sum1 = (uF[0][i].real)*Q[i]
    Ei = 0.5*4.*np.pi*332.064*(Sum1)
    print(i,Ei)
    q_uF = q_uF + Sum1

#Resultado de la energia de solvatacion total
E_Solv = 0.5*4.*np.pi*332.064*(q_uF)
print('Energia de Solvatacion: {:7.6f} [kCal/mol]'.format(E_Solv) )

end = time.time()
curr_time = (end - start)   
print("Tiempo total: {:5.2f} [s]".format(curr_time))

0 -24.298909893030633
1 -24.307086241942283
2 -24.306413031958584
Energia de Solvatacion: -72.912409 [kCal/mol]
Tiempo total: 21.63 [s]
